In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://hospital1-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


ModuleNotFoundError: No module named 'aries_basic_controller'

In [4]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener, message_listener], defaults=True)

## Copy Invite from Researcher

In [5]:
#Paste in invitation from researcher agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '31d22c96-b531-467a-8637-867f0e9199ac', 'label': 'Health Researcher', 'serviceEndpoint': 'http://172.17.0.1:8040', 'recipientKeys': ['Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE']}

In [6]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State invitation
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State request
Connection {'invitation_key': 'Hu8va1kwBxycxKa731q6FeenAmhk5s3x2X3MXXR7CckE', 'state': 'request', 'connection_id': 'ec35bfc5-68ef-45c4-87c6-5d9f63819c77', 'routing_state': 'none', 'updated_at': '2021-01-23 14:22:15.871577Z', 'initiator': 'external', 'invitation_mode': 'once', 'created_at': '2021-01-23 14:22:15.851490Z', 'accept': 'manual', 'their_label': 'Health Researcher', 'my_did': 'ALrPXCqEuqa5wYFUDDkfeo', 'request_id': 'b9560178-dc00-4496-9a8c-2da146166b4d'}
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State response
Connection Handler Called
Connection ec35bfc5-68ef-45c4-87c6-5d9f63819c77 in State active


In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

In [6]:
# Data pre-processing

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# prep
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler

#Read in Data
train_df = pd.read_csv('hospital1.csv')


########## START DATA CLEANING ###############


#dealing with missing data
#Let’s get rid of the variables "Timestamp",“comments”, “state” just to make our lives easier.
train_df = train_df.drop(['comments'], axis= 1)
train_df = train_df.drop(['state'], axis= 1)
train_df = train_df.drop(['Timestamp'], axis= 1)

# Assign default values for each data type
defaultInt = 0
defaultString = 'NaN'
defaultFloat = 0.0

# Create lists by data tpe
intFeatures = ['Age']
stringFeatures = ['Gender', 'Country', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                     'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                     'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                     'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                     'seek_help']
floatFeatures = []

# Clean the NaN's
for feature in train_df:
    if feature in intFeatures:
        train_df[feature] = train_df[feature].fillna(defaultInt)
    elif feature in stringFeatures:
        train_df[feature] = train_df[feature].fillna(defaultString)
    elif feature in floatFeatures:
        train_df[feature] = train_df[feature].fillna(defaultFloat)
    else:
        print('Error: Feature %s not recognized.' % feature)

#clean 'Gender'
#Slower case all columm's elements
gender = train_df['Gender'].str.lower()
#print(gender)

#Select unique elements
gender = train_df['Gender'].unique()

#Made gender groups
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in train_df.iterrows():

    if str.lower(col.Gender) in male_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

#Get rid of bullshit
stk_list = ['A little about you', 'p']
train_df = train_df[~train_df['Gender'].isin(stk_list)]

#complete missing age with mean
train_df['Age'].fillna(train_df['Age'].median(), inplace = True)

# Fill with media() values < 18 and > 120
s = pd.Series(train_df['Age'])
s[s<18] = train_df['Age'].median()
train_df['Age'] = s
s = pd.Series(train_df['Age'])
s[s>120] = train_df['Age'].median()
train_df['Age'] = s

#Ranges of Age
train_df['age_range'] = pd.cut(train_df['Age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

#There are only 0.20% of self work_interfere so let's change NaN to "Don't know
#Replace "NaN" string from defaultString

train_df['work_interfere'] = train_df['work_interfere'].replace([defaultString], 'Don\'t know' )

#Encoding data
labelDict = {}
for feature in train_df:
    le = preprocessing.LabelEncoder()
    le.fit(train_df[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    train_df[feature] = le.transform(train_df[feature])
    # Get labels
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] =labelValue

#Get rid of 'Country'
train_df = train_df.drop(['Country'], axis= 1)

# Scaling Age
scaler = MinMaxScaler()
train_df['Age'] = scaler.fit_transform(train_df[['Age']])

# define X and y
feature_cols = ['Age', 'Gender', 'family_history', 'benefits', 'care_options', 'anonymity', 'leave', 'work_interfere']
X = train_df[feature_cols]
y = train_df.treatment

# split X and y into training and testing sets
X_train, y_train = X, y

# Transform pandas dataframe to torch tensor for DL

x_train_data = torch.from_numpy(X_train.values)
x_train_data = x_train_data.float()

y_train_data = []
for data in y_train.values:
    y_train_data.append([data])
y_train_data = torch.tensor(y_train_data).float()

len(y_train_data)

333

In [7]:
import torch
import traceback

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

# Receive model from the Researcher and train it

model_dir = os.getcwd() + "/../model.pt"

print(model_dir)

# Pull in model
try:
    model = torch.load(model_dir)
except Exception as e:
    print("HOSPITAL FAILED TO LOAD MODEL")
    print("Exception Value: ",e)
    print("Traceback ",traceback.format_exc())
#     return False

print("HOSPITAL MODEL LOADED")


# Training Logic
print("HOSPITAL IS TRAINING")

# Define Optimizer
opt = optim.SGD(params=model.parameters(), lr=0.1)
 
# opt = torch.optim.SGD(model.parameters(), lr=0.05)


# Apply Differential Privacy

#privacy_engine = PrivacyEngine(model, batch_size=333, sample_size=1000, alphas=[10, 100], 
#                            noise_multiplier=1.3, max_grad_norm=1.0)

# privacy_engine.attach(opt)

for iter in range(50000):

    # 1) erase previous gradients (if they exist)
    opt.zero_grad()
    # log_msg("TRAIN DATA", x_train_data)

    # 2) make a prediction
    pred = model(x_train_data)

    # 3) calculate how much we missed
    loss = (((y_train_data - pred) ** 2).sum()) / len(x_train_data)

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) log_msg our progress
    if (iter % 5000 == 0):
        print("loss at epoch ", iter, ": ", loss.data)

torch.save(model, "../trained_model.pt")

# Detach the Differential Privacy library from the Optimizer (We may don't need this at all)
# privacy_engine.detach()



/Users/pavlito/PyDentity/projects/aries-fl/notebooks/hospital/../model.pt
HOSPITAL MODEL LOADED
HOSPITAL IS TRAINING
loss at epoch  0 :  tensor(0.2684)
loss at epoch  5000 :  tensor(0.1264)
loss at epoch  10000 :  tensor(0.1152)
loss at epoch  15000 :  tensor(0.1114)
loss at epoch  20000 :  tensor(0.1083)
loss at epoch  25000 :  tensor(0.1058)
loss at epoch  30000 :  tensor(0.1040)
loss at epoch  35000 :  tensor(0.1023)
loss at epoch  40000 :  tensor(0.1008)
loss at epoch  45000 :  tensor(0.0996)


In [ ]:
response = await agent_controller.terminate()
print(response)